Clustering Description, not based on visual categories
===============================

In [33]:
import numpy as np
from collections import Counter, defaultdict
import csv
from scipy.stats import entropy
import multiprocessing as mp
from functools import partial
import pickle
import gzip
from heapq import *
import json
import sys

In [34]:
# Load files
catNameFile = 'data/categoryKey.csv'
catName = {}
with open(catNameFile) as fin:
    r = csv.reader(fin)
    for row in r:
        catName[int(row[0]) - 1] = row[1]
numCat = len(catName)
print range(numCat) == catName.keys()

inputFile = 'data/cat_desc_wn.csv'
headers = None
descCatCntRaw = defaultdict(lambda :np.zeros(numCat))
# catCnt = np.zeros(numCat)
with open(inputFile) as fin:
    r = csv.reader(fin)
    for row in r:
        if headers is None:
            headers = row
            print headers
        else:
            cat, desc, cnt = row
            cat = int(cat) - 1
            cnt = int(cnt)
            descCatCntRaw[desc][cat] += cnt
            catCnt[cat] += cnt
descCnt = Counter()
for desc in descCatCnt:
    descCatCnt[desc] /= catCnt
    descCnt[desc] = np.sum(descCatCnt[desc])
print len(descCatCnt)

True
['category_id', 'description', 'count']
3058


In [35]:
def descFilter(topNum=1000):
    return [w for w, v in descCnt.most_common(topNum)]

In [36]:
def mergeCandidiate(nodeInfo, s1, s2):
    return (entropy(nodeInfo[s1][4] + nodeInfo[s2][4]),
#             - (nodeInfo[s1][5] + nodeInfo[s2][5]) / (nodeInfo[s1][3] + nodeInfo[s2][3]), 
           s1, s2)
    
def updateCandidate(nodeInfo, newS, activeNodes, candidatePool):
    for s2 in activeNodes:
        heappush(candidatePool, mergeCandidiate(nodeInfo, newS, s2))
    activeNodes.add(newS)

def chooseCandidate(nodeInfo, candidatePool, activeNodes, newId):
    jsd, s1, s2 = heappop(candidatePool)
    while s1 not in activeNodes or s2 not in activeNodes:
        jsd, s1, s2 = heappop(candidatePool)
    nodeInfo.append((newId, s1, s2, nodeInfo[s1][3] + nodeInfo[s2][3], nodeInfo[s1][4] + nodeInfo[s2][4],
                    nodeInfo[s1][5] + nodeInfo[s2][5], jsd))
    activeNodes.remove(s1)
    activeNodes.remove(s2)
    
def descHC(descList):
    leafNum = len(descList)
    activeNodes = set()
    nodeInfo = []
    candidatePool = []
    heapify(candidatePool)
    
    for leaf in xrange(leafNum):
        desc = descList[leaf]
        nodeInfo.append((leaf, None, None, descCnt[desc], descCatCnt[desc], entropy(descCatCnt[desc]), 0.0))
        updateCandidate(nodeInfo, leaf, activeNodes, candidatePool)
        if leaf % 100 == 0:
            print 'Adding leaf #', leaf
    
    nextId = leafNum
    while len(activeNodes) > 1:
        chooseCandidate(nodeInfo, candidatePool, activeNodes, nextId)
        updateCandidate(nodeInfo, nextId, activeNodes, candidatePool)
        nextId += 1
        if len(activeNodes) % 100 == 0:
            print 'Remaining active nodes #', len(activeNodes)
    return nodeInfo

In [37]:
# from scipy.stats import entropy # can't handle entropy(np.array([0, 1, 1]), np.array([0, 1, 1]))\n
from numpy.linalg import norm
def JSD(P, Q):
    _P = P / float(norm(P, ord=1))
    _Q = Q / float(norm(Q, ord=1))
    _M = 0.5 * (_P + _Q)
    return entropy(_M) - 0.5 * (entropy(_P) + entropy(_Q))

def nameCluster(descList, nodeInfo):
    leafNum = len(descList)
    totalNodes = len(nodeInfo)
    namingCandidate = []
    for desc in descList:
        namingCandidate.append(set([desc]))
    for p in xrange(leafNum, totalNodes):
        s1, s2 = nodeInfo[p][1], nodeInfo[p][2]
        namingCandidate.append(namingCandidate[s1] | namingCandidate[s2])
    bestNaming = []
    for i in xrange(totalNodes):
        cand = namingCandidate[i]
        tmp = np.zeros(numCat)
        for c in cand:
            tmp += descCatCnt[c]
        if i < leafNum:
            bestNaming.append(descList[i])
        else:
            bestNaming.append(min(cand, key=lambda w: JSD(nodeInfo[i][4], descCatCnt[w])))
        if i % 100 == 0:
            print 'Finding best Naming #', i
    return bestNaming

In [38]:
def recur_flattenTree(nodeInfo, bestNaming, usedWords, nodeList, root, parent, level):
    s1, s2 = nodeInfo[root][1], nodeInfo[root][2]
#     print root, s1, s2, parent, level
    if bestNaming[root] in usedWords:
        if s1 is None and s2 is None:
            return ()
        else:
            st1 = recur_flattenTree(nodeInfo, bestNaming, usedWords, nodeList, s1, parent, level)
            st2 = recur_flattenTree(nodeInfo, bestNaming, usedWords, nodeList, s2, parent, level)
            return st1 + st2
    else:
        nm = bestNaming[root]
        usedWords.add(nm)
        if s1 is None and s2 is None:
            nodeList.append((nm, parent, (), level))
            return ({'name':nm, 'children':()},)
        else:
            st1 = recur_flattenTree(nodeInfo, bestNaming, usedWords, nodeList, s1, nm, level + 1)
            st2 = recur_flattenTree(nodeInfo, bestNaming, usedWords, nodeList, s2, nm, level + 1)
            sons = st1 + st2
            nodeList.append((nm, parent, tuple(s['name'] for s in sons), level))
            return ({'name':nm, 'children':sons},)

def flattenTree(nodeInfo, bestNaming, descList):
    nodeList = []
    usedWords = set()
    tree = recur_flattenTree(nodeInfo, bestNaming, usedWords, nodeList, len(nodeInfo) - 1, None, 0)
    if set(descList) != usedWords or len(tree) != 1:
        raise RuntimeError('Something is wrong @ flattenTree!')
    return tree[0], nodeList

In [39]:
webpageFormat = 'vis/sample.html'
with open(webpageFormat) as fin:
    webpageString = fin.read()
def visTree(tree, treeSize, name):
    with open('vis/data/' + name + '.json', 'w') as fout:
        json.dump(tree, fout)
    with open('vis/' + name + '.html', 'w') as fout:
        print >>fout, webpageString.replace('##PATH_TO_TREE_JSON##', 'data/' + name + '.json')\
                        .replace('##HEIGHT##', str(10 * treeSize))

In [40]:
def buildDescTaxo(topNum):
    sys.setrecursionlimit(topNum * 2)
    descList = descFilter(topNum)
    nodeInfo = descHC(descList)
    bestNaming = nameCluster(descList, nodeInfo)
    tree, treeNodes = flattenTree(nodeInfo, bestNaming, descList)
    name = 'descTaxo_' + str(topNum)
    visTree(tree, len(treeNodes), name)
    with gzip.open('model/' + name + '.pickle.gz', 'wb') as fout:
        pickle.dump(treeNodes, fout)
    print 'Finished', topNum

In [41]:
# topNum = 500
# sys.setrecursionlimit(topNum * 2)
# descList = descFilter(topNum)
# nodeInfo = descHC(descList)
# bestNaming = nameCluster(descList, nodeInfo)
# tree, treeNodes = flattenTree(nodeInfo, bestNaming, descList)
# name = 'descTaxo_' + str(topNum)
# visTree(tree, len(treeNodes), name)
# with gzip.open('model/' + name + '.pickle.gz', 'wb') as fout:
#     pickle.dump(treeNodes, fout)
# print 'Finished', topNum

In [43]:
buildDescTaxo(1200)

Adding leaf # 0
Adding leaf # 100
Adding leaf # 200
Adding leaf # 300
Adding leaf # 400
Adding leaf # 500
Adding leaf # 600
Adding leaf # 700
Adding leaf # 800
Adding leaf # 900
Adding leaf # 1000
Adding leaf # 1100
Remaining active nodes # 1100
Remaining active nodes # 1000
Remaining active nodes # 900
Remaining active nodes # 800
Remaining active nodes # 700
Remaining active nodes # 600
Remaining active nodes # 500
Remaining active nodes # 400
Remaining active nodes # 300
Remaining active nodes # 200
Remaining active nodes # 100
Finding best Naming # 0
Finding best Naming # 100
Finding best Naming # 200
Finding best Naming # 300
Finding best Naming # 400
Finding best Naming # 500
Finding best Naming # 600
Finding best Naming # 700
Finding best Naming # 800
Finding best Naming # 900
Finding best Naming # 1000
Finding best Naming # 1100
Finding best Naming # 1200
Finding best Naming # 1300
Finding best Naming # 1400
Finding best Naming # 1500
Finding best Naming # 1600
Finding best Nam